In [1]:
import numpy as np
import random
import math
import pandas as pd
from random import sample

In [2]:
def euc_2d(c1,c2):
    return math.sqrt((c1[0]-c2[0])**2+(c1[1]-c2[1])**2)

def random_permutation(cities):
    perm = np.random.permutation(len(cities))
    return perm

def stochastic_two_opt(permutation):
    perm_size = len(permutation)
    new_perm = []
    x = list(permutation)
    start,end = sorted(sample(range(perm_size),2))
    exclude = [start]
    if start==0:
        exclude.append(perm_size-1)
    else:
        exclude.append(start-1)
    if start==perm_size-1:
        exclude.append(0)
    else:
        exclude.append(start+1)
    while end in exclude:
        end = sample(range(perm_size),1)[0]
    if end < start:
        start,end = end,start
    new_perm = x[:start]
    new_perm += (list(reversed(x[start:end])))
    new_perm += x[end:]
    
    return new_perm

def augmented_cost(permutation, penalties, cities,lambda_):
    distance, augmented = 0, 0
    for i,c1 in enumerate(permutation):
        if i == len(permutation)-1:
            c2 = permutation[0]
        else:
            c2 = permutation[i+1]
        if c1 > c2 :
            c1,c2 = c2,c1
        d = euc_2d(cities[c1],cities[c2])
        distance += d
        augmented += d+(lambda_ * (penalties[c1][c2]))
    return [distance,augmented]

def cost(cand, penalties, cities, lambda_):
    cost, acost = augmented_cost(cand['vector'], penalties, cities, lambda_)
    cand['cost'], cand['aug_cost'] = cost, acost

def local_search(current, cities, penalties, max_no_improv, lambda_):
    cost(current, penalties, cities, lambda_)
    count = 0
    while count < max_no_improv:
        candidate = {'vector':stochastic_two_opt(current['vector'])}
        cost(candidate,penalties,cities,lambda_)
        if candidate['aug_cost'] < current['aug_cost']:
            count = 0 
        else:
            count += 1
        if candidate['aug_cost'] < current['aug_cost']:
            current = candidate           
    return current

def calculate_feature_utilities(penal, cities, permutation):
    utilities = [0] * len(permutation)
    for i,c1 in enumerate(permutation):
        if i == len(permutation)-1:
            c2 = permutation[0]
        else:
            c2 = permutation[i+1]
        if c1 > c2 :
            c1,c2 = c2,c1
        utilities[i] = euc_2d(cities[c1],cities[c2])/(1+penal[c1][c2])
    return utilities

def update_penalties(penalties, cities, permutation, utilities):
    max_ = max(utilities)
    for i,c1 in enumerate(permutation):
        if i == len(permutation)-1:
            c2 = permutation[0]
        else:
            c2 = permutation[i+1]
        if c1 > c2 :
            c1,c2 = c2,c1
        if utilities[i] == max_:
            penalties[c1][c2] += 1 
    return penalties

def search(max_iterations, cities, max_no_improv, lambda_):
    current = {'vector':random_permutation(cities)}
    best = current
    penalties = [ [0] * len(cities) for i in range(len(cities))]
    n = 0
    while n < max_iterations:
        current = local_search(current,cities,penalties,max_no_improv,lambda_)
        utilities = calculate_feature_utilities(penalties,cities,current['vector'])
        update_penalties(penalties,cities,current['vector'],utilities)
        if current['cost'] < best['cost']:
            best = current
            print("Best solution of iters_ {} is_ {}".format(n,current['cost']))
        n +=1
    return best

In [3]:
# problem configuration
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]
# algorithm configuration
max_iterations = 150
max_no_improv = 200
alpha = 0.3
local_search_optima = 12000.0
lambda_ = alpha * (local_search_optima/float(len(berlin52)))
# execute the algorithm
# best = search(max_iterations, berlin52, max_no_improv, lambda)

In [4]:
search(max_iterations, berlin52, max_no_improv, lambda_)

Best solution of iters_ 0 is_ 8196.684464239215
Best solution of iters_ 1 is_ 8112.505836471694
Best solution of iters_ 4 is_ 8101.001297431268
Best solution of iters_ 5 is_ 7932.154423688871
Best solution of iters_ 9 is_ 7905.263813338702
Best solution of iters_ 16 is_ 7904.96750399116
Best solution of iters_ 54 is_ 7903.868683349037
Best solution of iters_ 65 is_ 7890.893189891068
Best solution of iters_ 94 is_ 7808.910150856435
Best solution of iters_ 101 is_ 7804.3145297777355


{'vector': [41,
  1,
  6,
  16,
  2,
  44,
  18,
  40,
  7,
  8,
  9,
  42,
  32,
  50,
  10,
  51,
  12,
  13,
  46,
  25,
  26,
  27,
  11,
  24,
  3,
  5,
  14,
  4,
  23,
  47,
  45,
  15,
  43,
  33,
  36,
  37,
  39,
  38,
  35,
  34,
  48,
  31,
  0,
  21,
  17,
  30,
  20,
  22,
  19,
  49,
  28,
  29],
 'cost': 7804.3145297777355,
 'aug_cost': 10019.699145162354}